# 1. Carga y Limpieza del texto

In [1]:
# Leer el archivo
with open("Gutenberg.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Eliminar encabezados/pies del archivo de Gutenberg
start = text.find("*** START OF THE PROJECT GUTENBERG EBOOK")
end = text.find("*** END OF THE PROJECT GUTENBERG EBOOK")
if start != -1 and end != -1:
    text = text[start:end]

# Limpiar saltos de línea dobles y espacios extra
text = text.replace('\r', '').replace('\n\n', '\n').strip()
print(f"Longitud del texto: {len(text)} caracteres")


Longitud del texto: 5318165 caracteres


# 2. Preprocesamiento a nivel de caracter

In [2]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Crear vocabulario
chars = sorted(set(text))
char2idx = {c: i for i, c in enumerate(chars)}
idx2char = {i: c for c, i in char2idx.items()}

vocab_size = len(chars)
print(f"Vocabulario de {vocab_size} caracteres")

# Convertir texto a enteros
encoded = [char2idx[c] for c in text]


Vocabulario de 100 caracteres


# 3. Crear secuencias de entrenamiento

In [3]:
sequence_length = 40
step = 1
X = []
y = []

for i in range(0, len(encoded) - sequence_length, step):
    X.append(encoded[i:i + sequence_length])
    y.append(encoded[i + sequence_length])

X = np.array(X)
y = to_categorical(y, num_classes=vocab_size)

print(f"Total de secuencias: {len(X)}")


Total de secuencias: 5318125


# 4. Modelo RNN con LSTM

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=50, input_length=sequence_length))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.build(input_shape=(None, sequence_length))
model.summary()


c:\Users\gerar\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 40, 50)         │         5,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │        91,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        12,900 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,548 (427.92 KB)

 Trainable params: 109,548 (427.92 KB)

 Non-trainable params: 0 (0.00 B)

# 5. Entrenamiento

In [6]:
model.fit(X, y, batch_size=64, epochs=20)

Epoch 1/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2780s 33ms/step - loss: 1.9811
Epoch 2/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2744s 33ms/step - loss: 1.5972
Epoch 3/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2797s 33ms/step - loss: 1.5434
Epoch 4/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2745s 33ms/step - loss: 1.5170
Epoch 5/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2741s 33ms/step - loss: 1.5009
Epoch 6/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2776s 33ms/step - loss: 1.4911
Epoch 7/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2795s 34ms/step - loss: 1.4823
Epoch 8/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2803s 34ms/step - loss: 1.4767
Epoch 9/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2812s 34ms/step - loss: 1.4712
Epoch 10/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2841s 34ms/step - loss: 1.4681
Epoch 11/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2864s 34ms/step - loss: 1.4655
Epoch 12/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2884s 35ms/step - loss: 1.4614
Epoch 13/20
83096/83096 ━━━━━━━━━━━━━━━━━━━━ 2911s 35ms/step - loss: 1.4594
Epoch 14/20
83096/830

In [7]:
model.save("poem_generator.h5")

# 6. Funcion para muestreo con temperatura

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)


# 7. Generacion de Texto

In [12]:
def generate_text(seed, length=400, temperature=1.0, force_end_punctuation=True):
    generated = seed
    seed_encoded = [char2idx[c] for c in seed]

    for _ in range(length):
        padded = pad_sequences([seed_encoded[-sequence_length:]], maxlen=sequence_length)
        preds = model.predict(padded, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = idx2char[next_index]
        generated += next_char
        seed_encoded.append(next_index)

    if force_end_punctuation:
        # Buscar desde el final hacia atrás el primer '.', '?', o '!'
        for i in range(len(generated)-1, -1, -1):
            if generated[i] in ['.', '!', '?']:
                generated = generated[:i+1]
                break
        else:
            # Si no se encontró puntuación final, agregamos '.'
            generated += '.'

    return generated


# 8. Prueba de Generacion

In [14]:
print(generate_text("The sun ", length=400, temperature=1.0))

The sun about
Wourrion a grown doth barge did, ’righs,
And do him a whomp, to speak for it again for thee.
Go Olivy.
SECOND GENTLEMAN.
Tillance upon thy Richard, y’ell in grave?
LODin Firief’s Mortives. Arterp, and pleat me
These death yet grurd on me and day,
As thy most hearts
As i’ th’ dead’s all. No, and the hand?
LAOL.
